In [ ]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

def fetch_gnomad_variants(chrom, start, stop, referenceGenome='GRCh38', dataset='gnomad_r4'):
    url = 'https://gnomad.broadinstitute.org/api'
    query = '''
    query($chrom: String!, $start: Int!, $stop: Int!, $referenceGenome: ReferenceGenomeId!, $dataset: DatasetId!) {
      region(chrom: $chrom, start: $start, stop: $stop, reference_genome: $referenceGenome) {
        variants(dataset: $dataset) {
          variantId
          chrom
          pos
          ref
          alt
          genome {
            af
          }
          consequence
        }
      }
    }
    '''
    variables = {
        "chrom": chrom,
        "start": start,
        "stop": stop,
        "referenceGenome": referenceGenome,
        "dataset": dataset
    }
    response = requests.post(url, json={'query': query, 'variables': variables})
    data = response.json()
    if 'errors' in data:
        raise Exception(f"GraphQL error: {data['errors']}")
    variants = data['data']['region']['variants']
    return variants

def variants_to_dataframe(variants):
    records = []
    for v in variants:
        af = v['genome']['af'] if v['genome'] else 0.0
        cons = v['consequence'] if v['consequence'] else 'unknown'
        records.append({
            'variantId': v['variantId'],
            'chrom': v['chrom'],
            'pos': v['pos'],
            'ref': v['ref'],
            'alt': v['alt'],
            'af': af,
            'consequence': cons
        })
    df = pd.DataFrame(records)
    return df

def create_plots(df, bin_size=100):
    min_pos = df['pos'].min()
    max_pos = df['pos'].max()
    bins = np.arange(min_pos, max_pos + bin_size, bin_size)
    df['pos_bin'] = pd.cut(df['pos'], bins=bins, right=False)
    grouped = df.groupby(['consequence', 'pos_bin']).agg(sum_af=('af', 'sum')).reset_index()
    grouped['bin_left'] = grouped['pos_bin'].apply(lambda x: x.left)

    consequences = grouped['consequence'].unique()
    color_map = px.colors.qualitative.T10
    color_dict = {cons: color_map[i % len(color_map)] for i, cons in enumerate(consequences)}

    # Линейный график с легендой
    fig_line = go.Figure()
    for cons in consequences:
        data = grouped[grouped['consequence'] == cons].sort_values('bin_left')
        fig_line.add_trace(go.Scatter(
            x=data['bin_left'],
            y=data['sum_af'],
            mode='lines+markers',
            name=cons,
            fill='tozeroy',
            line=dict(color=color_dict[cons]),
            marker=dict(color=color_dict[cons], line=dict(width=0)),
            showlegend=True
        ))

    fig_line.update_layout(
        title="Sum AF by consequence and genomic position bins",
        xaxis_title="Genomic position (left boundary of bin)",
        yaxis_title="Sum AF",
        hovermode='x unified',
        legend=dict(
            x=1.05,
            y=0.5,
            bgcolor='rgba(0,0,0,0)',
            borderwidth=0,
            font=dict(size=12),
            traceorder='normal',
            itemsizing='constant'
        ),
        margin=dict(r=180),
        xaxis=dict(
            tickformat='d',
            showgrid=True
        ),
        yaxis=dict(showgrid=True)
    )

    # Подготовка данных для pie с теми же цветами
    consequence_counts = df['consequence'].value_counts()
    pie_colors = [color_dict[cons] for cons in consequence_counts.index]

    fig_pie = px.pie(
        names=consequence_counts.index,
        values=consequence_counts.values,
        color=consequence_counts.index,
        color_discrete_map=color_dict,
        title=None  # заголовок будет отдельным html-элементом
    )
    fig_pie.update_traces(
        textposition='outside',
        textinfo='percent+label',
        pull=[0.05] * len(consequence_counts),
        automargin=True,
        insidetextorientation='radial',
        marker=dict(line=dict(color='rgba(0,0,0,0)', width=0))
    )

    # Убираем легенду из pie
    fig_pie.update_layout(
        showlegend=False,
        margin=dict(l=20, r=20, t=40, b=20),
        uniformtext_minsize=10,
        uniformtext_mode='hide',
    )

    return fig_line, fig_pie

def main():
    print("Введите параметры для запроса к gnomAD (нажмите Enter для значения по умолчанию):")
    chrom = input("Chromosome (default 19): ") or "19"
    start = input("Start position (default 11110000): ") or "11110000"
    stop = input("Stop position (default 11125801): ") or "11125801"
    bin_size = input("Bin size (default 100): ") or "100"
    referenceGenome = input("Reference genome (default GRCh38): ") or "GRCh38"
    dataset = input("Dataset (default gnomad_r4): ") or "gnomad_r4"

    try:
        start = int(start)
        stop = int(stop)
        bin_size = int(bin_size)
    except ValueError:
        print("Ошибка: start, stop и bin size должны быть числами.")
        return

    print(f"Запрос: chrom={chrom}, start={start}, stop={stop}, bin_size={bin_size}, refGenome={referenceGenome}, dataset={dataset}")

    print("Получаем варианты...")
    variants = fetch_gnomad_variants(chrom, start, stop, referenceGenome, dataset)
    print(f"Получено вариантов: {len(variants)}")

    df = variants_to_dataframe(variants)

    print("Создаём графики...")
    fig_line, fig_pie = create_plots(df, bin_size=bin_size)

    html_filename = "gnomad_plots.html"
    with open(html_filename, "w", encoding="utf-8") as f:
        f.write(f"""
<html>
<head>
    <title>gnomAD Variant Plots</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
        }}
    </style>
</head>
<body>
    <h1 style="text-align:center;">gnomAD Variant Visualization</h1>
    <div style="display:flex; align-items:center; justify-content:center; gap:40px;">
        <div style="width:500px; height:500px; position: relative;">
            <div style="position:absolute; top:0; left:50%; transform: translateX(-50%); font-weight:bold; font-size:18px;">
                Distribution of consequences
            </div>
            <div id="pie_chart" style="width:100%; height:100%;"></div>
        </div>
        <div id="line_plot" style="width:800px; height:600px;"></div>
    </div>
    <script>
        var line_fig = {fig_line.to_json()};
        var pie_fig = {fig_pie.to_json()};
        Plotly.newPlot('line_plot', line_fig.data, line_fig.layout);
        Plotly.newPlot('pie_chart', pie_fig.data, pie_fig.layout);
    </script>
</body>
</html>
""")
    print(f"Графики сохранены в файл {html_filename}. Откройте его в браузере.")

if __name__ == "__main__":
    main()

